In [1]:
import itertools
import os
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_addons as tfa
 
print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

TF version: 2.6.0
Hub version: 0.8.0
GPU is available


In [2]:
model_name = "efficientnetv2-s-21k" 
model_handle_map = {
  "efficientnetv2-s-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_s/feature_vector/2",
  "efficientnetv2-m-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_m/feature_vector/2",
  "efficientnetv2-l-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_l/feature_vector/2",
  "efficientnetv2-xl-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_xl/feature_vector/2",
  "efficientnetv2-b0-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b0/feature_vector/2",
  "efficientnetv2-b1-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b1/feature_vector/2",
  "efficientnetv2-b2-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b2/feature_vector/2",
  "efficientnetv2-b3-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b3/feature_vector/2",
}

model_image_size_map = {
  "efficientnetv2-s-21k": 384,
  "efficientnetv2-m-21k": 480,
  "efficientnetv2-l-21k": 480,
  "efficientnetv2-xl-21k": 512,
  "efficientnetv2-b0-21k": 224,
  "efficientnetv2-b1-21k": 240,
  "efficientnetv2-b2-21k": 260,
  "efficientnetv2-b3-21k": 300,
  "efficientnetv2-s-21k-ft1k": 384,
  "efficientnetv2-m-21k-ft1k": 480,
  "efficientnetv2-l-21k-ft1k": 480,
  "efficientnetv2-xl-21k-ft1k": 512,
  "efficientnetv2-b0-21k-ft1k": 224,
  "efficientnetv2-b1-21k-ft1k": 240,
  "efficientnetv2-b2-21k-ft1k": 260,
  "efficientnetv2-b3-21k-ft1k": 300, 
}

model_handle = model_handle_map.get(model_name)
pixels = model_image_size_map.get(model_name, 224)
print(f"Selected model: {model_name} : {model_handle}")
IMAGE_SIZE = (pixels, pixels)
print(f"Input size {IMAGE_SIZE}")
BATCH_SIZE = 12

Selected model: efficientnetv2-s-21k : https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_s/feature_vector/2
Input size (384, 384)


In [3]:
data_dir = ("./data/")
def build_dataset(subset):
  return tf.keras.preprocessing.image_dataset_from_directory(
      data_dir,
      validation_split=.10,
      subset=subset,
      label_mode="categorical",
      shuffle = True,
      # Seed needs to provided when using validation_split and shuffle = True.
      # A fixed seed is used so that the validation set is stable across runs.
      seed=123,
      image_size=IMAGE_SIZE,
      batch_size=1)

train_ds = build_dataset("training")
class_names = tuple(train_ds.class_names)
train_size = train_ds.cardinality().numpy()
train_ds = train_ds.unbatch().batch(BATCH_SIZE)
train_ds = train_ds.repeat()

normalization_layer = tf.keras.layers.Rescaling(1. / 255)
preprocessing_model = tf.keras.Sequential([normalization_layer])
do_data_augmentation = True
if do_data_augmentation:
  preprocessing_model.add(
      tf.keras.layers.RandomFlip(mode="horizontal"))

train_ds = train_ds.map(lambda images, labels:
                        (preprocessing_model(images), labels))

val_ds = build_dataset("validation")
valid_size = val_ds.cardinality().numpy()
val_ds = val_ds.unbatch().batch(BATCH_SIZE)
val_ds = val_ds.map(lambda images, labels:
                    (normalization_layer(images), labels))

Found 80270 files belonging to 14 classes.
Using 72243 files for training.
Found 80270 files belonging to 14 classes.
Using 8027 files for validation.


In [4]:
do_fine_tuning = True
print("Building model with", model_handle)
model = tf.keras.Sequential([
    # Explicitly define the input shape so the model can be properly
    # loaded by the TFLiteConverter
    tf.keras.layers.InputLayer(input_shape=IMAGE_SIZE + (3,)),
    hub.KerasLayer(model_handle, trainable=do_fine_tuning),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(len(class_names),
                          kernel_regularizer=tf.keras.regularizers.l2(0.00001))
])
model.build((None,)+IMAGE_SIZE+(3,))
model.summary()

Building model with https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_s/feature_vector/2
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              20331360  
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 14)                17934     
Total params: 20,349,294
Trainable params: 20,195,422
Non-trainable params: 153,872
_________________________________________________________________


In [5]:
class_weight = {0: 3.1045,
                1: 1,
                2: 6.3571,
                3: 1.3096,
                4: 6.4676,
                5: 1.6541,
                6: 2.6821,
                7: 1.2028,
                8: 1.5817,
                9: 7.5519,
                10: 1.5921,
                11: 2.2272,
                12: 5.7142,
                13: 8.4212}

In [6]:
lr_decayed_fn = (tf.keras.optimizers.schedules.CosineDecayRestarts(initial_learning_rate=0.01,t_mul=2.0,m_mul=1,first_decay_steps=1000,alpha=0.0))
model.compile(
  optimizer=tf.keras.optimizers.SGD(learning_rate=lr_decayed_fn, momentum=0.9), 
  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
  metrics=['accuracy'])

In [7]:
steps_per_epoch = train_size // BATCH_SIZE
validation_steps = valid_size // BATCH_SIZE
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3,restore_best_weights=True)
hist = model.fit(
    train_ds,
    epochs=8, steps_per_epoch=steps_per_epoch,
    validation_data=val_ds,
    validation_steps=validation_steps,
    callbacks=[callback],
    class_weight=class_weight).history

Epoch 1/8
6020/6020 [==============================] - 2449s 402ms/step - loss: 1.8407 - accuracy: 0.9081 - val_loss: 0.6194 - val_accuracy: 0.9804
Epoch 2/8
6020/6020 [==============================] - 2392s 397ms/step - loss: 1.4465 - accuracy: 0.9679 - val_loss: 0.5972 - val_accuracy: 0.9845
Epoch 3/8
6020/6020 [==============================] - 2421s 402ms/step - loss: 1.3715 - accuracy: 0.9781 - val_loss: 0.6343 - val_accuracy: 0.9727
Epoch 4/8
6020/6020 [==============================] - 2401s 399ms/step - loss: 1.3484 - accuracy: 0.9811 - val_loss: 0.5815 - val_accuracy: 0.9885
Epoch 5/8
6020/6020 [==============================] - 2468s 410ms/step - loss: 1.2707 - accuracy: 0.9931 - val_loss: 0.5731 - val_accuracy: 0.9919
Epoch 6/8
6020/6020 [==============================] - 2438s 405ms/step - loss: 1.3328 - accuracy: 0.9824 - val_loss: 0.6236 - val_accuracy: 0.9742
Epoch 7/8
6020/6020 [==============================] - 2459s 408ms/step - loss: 1.3222 - accuracy: 0.9834 - val_

In [10]:
test_dir = ("./test/")
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    labels=None,
    color_mode='rgb',
    batch_size=1,
    image_size=(384, 384),
    shuffle=False,
    validation_split=None,
    subset=None,
    crop_to_aspect_ratio=False,
)
probability_model = tf.keras.Sequential([model,tf.keras.layers.Softmax()])
def process(image):
    image = tf.cast(image/255. ,tf.float32)
    return image
test_ds = test_ds.map(process)
predictions = probability_model.predict(test_ds)

Found 20000 files belonging to 1 classes.


In [9]:
a = np.zeros(20000)
cat = ["banana", "bareland", "carrot", "corn", "dragonfruit", "garlic", "guava", "peanut", "pineapple", "pumpkin", "rice", "soybean", "sugarcane", "tomato"]
for i in range(0,19999):
    a[i] = np.argmax(predictions[i])
os.remove("label.csv")
np.savetxt('label.csv', a,fmt='%d', delimiter=',')